In [1]:
import pandas as pd
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence

In [2]:
# import training and validation data
train_data = np.memmap('train.bin', dtype=np.uint16, mode='r')
val_data = np.memmap('val.bin', dtype=np.uint16, mode='r')
with open('encoder.pickle', 'rb') as f:
    encoder = pickle.load(f)
decoder = {v:k for k,v in encoder.items()}

### TODO:
* Transformer Model
    * update using https://github.com/karpathy/nanoGPT
  * old todo
    * fix the math error in hyperparameters - batch size =wrong= batch size * # X columns
    * figure out how to save, run, change lr, resume, etc
    * OPTIONAL: set up CUDA

### old transformer model (build by chatgpt / doesn't work)

In [3]:
raise ThisDoesntWorkYetException

NameError: name 'ThisDoesntWorkYetException' is not defined

In [ ]:
# Preprocessing

def zero_pad(lst, N=10):
    lst += [0] * (N - len(lst))
    return lst

def create_sliding_window_sequences(chord_progressions, window_size_min, window_size_max, zero_padding=10):
    X, y = [], []
    
    for window_size in range(window_size_min, window_size_max+1):
        for progression in chord_progressions:
            if len(progression) > window_size:
                for i in range(len(progression) - window_size):
                    seq = progression[i:i + window_size]
                    next_chord = progression[i + window_size]

                    X.append(zero_pad(seq, zero_padding))
                    y.append(next_chord)

    return X, y

X, y = create_sliding_window_sequences(chord_progressions['encoded'], 3, sequence_length-1, zero_padding=sequence_length)
len(X), len(y)

In [ ]:
# Transformer Model - PyTorch

# Create a PyTorch dataset
X_tensor = torch.LongTensor(X)  # X is a list of input sequences
y_tensor = torch.LongTensor(y)  # y is a list of target chords
dataset = TensorDataset(X_tensor, y_tensor)

# Create a PyTorch data loader
batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define model hyperparameters
d_model = 64  # Size of the model embedding
nhead = 4  # Number of attention heads
num_layers = 3  # Number of transformer layers
dim_feedforward = 128  # Size of the feedforward network
vocab_size = len(tokens)

# Define the model
class ChordTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers, dim_feedforward):
        super(ChordTransformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers, dim_feedforward)
        self.fc = nn.Linear(d_model, vocab_size)
        
    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x, x)
        x = self.fc(x)
        return x
    
# Estimate total memory used
total_memory_for_parameters = 2 * ((vocab_size * d_model * 4) + (d_model * vocab_size * 4))
f'{10 * total_memory_for_parameters * batch_size // (1024*1024) / 10} MB'

In [ ]:
# Initialize model, loss, and optimizer

model = ChordTransformer(len(encoder), d_model, nhead, num_layers, dim_feedforward)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
%%time

# Training loop with batch processing
epochs = 100
for epoch in range(epochs):
    for batch_idx, (X_batch, y_batch) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output.view(-1, len(encoder)), y_batch.view(-1))
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
